In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import geopandas as gpd
from shapely.geometry import Point

In [2]:
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd

In [3]:
import plotly.io as pio
pio.templates.default = "plotly"

In [4]:
# Importing datasets
df_cumulative_income = pd.read_csv('Cumulative_Income.csv')
df_Gini_Coefficient = pd.read_csv('Gini_Coefficient.csv')

In [5]:
df_Gini_Coefficient_2a = df_Gini_Coefficient.groupby('Region').mean();
df_Gini_Coefficient_2a.sort_values(by='Gini Coefficient');

In [6]:
# Importing datasets
df2 = pd.read_csv('df2.csv')

In [7]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [8]:
ph = gpd.read_file("gadm41_PHL_1.shp")
FIES = pd.read_excel('Table_1_1.xlsx', sheet_name=0)
FIES_1=pd.read_excel('Table_1_2.xlsx', sheet_name=0)
ph_new = ph.rename(columns={'NAME_1': 'Provinces'})
merged = ph_new.merge(FIES, on='Provinces')
merged_prov = merged.merge(FIES_1, on='Provinces')

In [9]:
prov_dash = merged_prov[['Income','Expenditure','AvgIncome','AvgExpenditure','GrowthRate_AveIncome', 'GrowthRate_AveExpenditure']]
metrics_option_1 = []
for item in prov_dash:
  metrics_option_1.append({
      'label': item,
      'value': item
  })

In [10]:
# Creating the app
px.set_mapbox_access_token("pk.eyJ1IjoiZmVyZGllMTIzNDUiLCJhIjoiY2xna3diYXQ0MDQ4bzNlcWl5Yzh4ajlxZyJ9.h00WfcnG_51aY_Ovxaw5Vw")

page = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

sorted = df_Gini_Coefficient_2a.sort_values(by='Gini Coefficient').index
region_dict = dict(zip(sorted,range(0,len(sorted))))
Average_Gini = round(df_Gini_Coefficient_2a['Gini Coefficient'].mean(),4)

page.layout = dbc.Container([
    # Header


    html.Nav(
    html.Ul([
        html.Li([
            html.A('Home', href='#', className='nav-link', style={'border': '1px solid grey'})
        ], className='nav-item'),
        html.Li([
            html.A('Understanding Poverty', href='#', className='nav-link', style={'border': '1px solid grey'})
        ], className='nav-item'),
        html.Li([
            html.A('Zooming in on Employment', href='#', className='nav-link', style={'border': '1px solid grey'})
        ], className='nav-item'),
        html.Li([
            html.A('Finding Opportunities', href='#', className='nav-link', style={'border': '1px solid grey'})
        ], className='nav-item')
    ], className='nav'),

),

  # Line Breaks

     html.Div([
          html.Br(), html.Br(), html.Br(), html.Br(),  html.Br(),
     ]),


    # Header 1
    html.Section([
        html.H2("Understanding Poverty", className="text-center mb-3"),
        html.H5("", className="text-center mb-5"),
    ]),

    html.Div([
      #html.H2("Philippine Income & Expenditure Distribution Choropleth Map"),
      dcc.Dropdown(id="choropleth-select", options=metrics_option_1,value='Income'),
      dcc.Graph(id="Map_01")
    ]),

    html.Div([
        html.Label("Select island group/s of interest"),
        dcc.Checklist(id='checklist_1',
                      options=['Luzon', 'Visayas', 'Mindanao'],
                      value=['Luzon', 'Visayas','Mindanao']),
        dcc.Graph(id='viz2a', animate = False)
    ]),

    # Div for Viz2b_TotalIncome
    html.Div([
        dcc.Graph(id='viz2b_TotalIncome', animate = False)
    ]),

    # Div for Viz2b_CumulativeIncome
    html.Div([
        dcc.Graph(id='viz2b_CumulativeIncome', animate = False)
    ]),


])

@page.callback(
    Output('Map_01', 'figure'),
    Input('choropleth-select', 'value')
)

def update_map_01(selected_metric_01):
    column = None

    if selected_metric_01 != None:
      column = selected_metric_01
      if column == 'GrowthRate_AveIncome' or column == 'GrowthRate_AveExpenditure':
          negative_color = np.array([252, 187, 129])
          middle_color = np.array([255, 255, 255])
          positive_color = np.array([154, 98, 164])

          min_value = merged_prov[column].min()
          max_value = merged_prov[column].max()

          num_colors_per_segment = 6

          negative_segment = [
              tuple(np.round(negative_color + i / (num_colors_per_segment - 1) * (middle_color - negative_color)).astype(int))
              for i in range(num_colors_per_segment)
          ]

          # Create the color scale for the positive segment (from white to #9a62a4)
          positive_segment = [
              tuple(np.round(middle_color + i / (num_colors_per_segment - 1) * (positive_color - middle_color)).astype(int))
              for i in range(1, num_colors_per_segment)
          ]

          # Combine both segments and convert the RGB values to strings
          color_scale = [f'rgb{color}' for color in negative_segment + positive_segment]
      elif column !='GrowthRate_AveIncome' or column !='GrowthRate_AveExpenditure':
          start_color = np.array([255, 255, 255])  # RGB values for white
          end_color = np.array([154, 98, 164])  # RGB values for #9a62a4
          num_colors = 11
          color_scale = [tuple(np.round(start_color + i / (num_colors - 1) * (end_color - start_color)).astype(int)) for i in range(num_colors)]
          color_scale = [f'rgb{color}' for color in color_scale]

    fig_map = px.choropleth_mapbox(merged_prov, geojson=merged_prov.geometry,
                                locations=merged_prov.index,
                                color=column,
                                color_continuous_scale=color_scale,
                                zoom=4.5,
                                mapbox_style='light',
                                center={"lat": 12.120649, "lon": 122.610799},
                                hover_data={'Provinces': True},
                                custom_data=['Provinces'])

    fig_map.update_traces(
          hovertemplate="<b>Region:</b> %{customdata[0]}<br><b>" + column + ":</b> %{z:.2f}<extra></extra>")

    fig_map.update_layout(height=600)

    return fig_map

# Updating the graph based on clickdata
@page.callback(
    Output("viz2b_TotalIncome", "figure"),
    [Input("viz2a", "clickData")]
)

def update_graph(click_data):

      if click_data is None:
          clicked_category = 'NCR'
          filtered_df = df2[df2['Region'] == clicked_category]
      else:
          clicked_category = click_data['points'][0]['y']
          filtered_df = df2[df2['Region'] == clicked_category]

      if clicked_category in ['REGION I - ILOCOS REGION',
                    'REGION II - CAGAYAN VALLEY',
                    'REGION III - CENTRAL LUZON',
                    'REGION IVA - CALABARZON',
                    'REGION V - BICOL REGION',
                    'NCR', 'CAR', 'MIMAROPA']:
                    color = '#fcb97d'
      elif clicked_category in ['REGION VI - WESTERN VISAYAS',
                      'REGION VII - CENTRAL VISAYAS',
                      'REGION VIII - EASTERN VISAYAS']:
                      color = '#a0b9c6'
      else:
          color = '#9a98b5'

      fig = px.bar(filtered_df,
                  x="Decile",
                  y="Total Income",
                  text="Total Income",
                  title=f"Total Income by Decile for {clicked_category}",
                  color_discrete_sequence=[color])

      return fig

@page.callback(
    Output("viz2b_CumulativeIncome", "figure"),
    [Input("viz2a", "clickData")]
)

def update_graph_1(click_data):

      if click_data is None:
          clicked_category = 'NCR'
          filtered_df_cumulative = df_cumulative_income[df_cumulative_income['Region'] == clicked_category]
      else:
          clicked_category = click_data['points'][0]['y']
          filtered_df_cumulative = df_cumulative_income[df_cumulative_income['Region'] == clicked_category]

      if clicked_category in ['REGION I - ILOCOS REGION',
                    'REGION II - CAGAYAN VALLEY',
                    'REGION III - CENTRAL LUZON',
                    'REGION IVA - CALABARZON',
                    'REGION V - BICOL REGION',
                    'NCR', 'CAR', 'MIMAROPA']:
                    color = '#fcb97d'
      elif clicked_category in ['REGION VI - WESTERN VISAYAS',
                      'REGION VII - CENTRAL VISAYAS',
                      'REGION VIII - EASTERN VISAYAS']:
                      color = '#a0b9c6'
      else:
          color = '#9a98b5'

      # Create a bar chart trace
      bar_trace = go.Bar(
          x=filtered_df_cumulative['Cumulative Population'],
          y=filtered_df_cumulative['Cumulative Income'],
          text=['{:.2f}%'.format(v) for v in filtered_df_cumulative['Cumulative Income']],
          name='Cumulative Income',
          hovertemplate='Cumulative Population: %{x:.2f}%<br>Cumulative Income: %{y:.2f}%<extra></extra>', # Custom hover template
          textposition='outside',
          textfont={
              'size': 14
          },
          insidetextanchor='start',
          marker = {'color':color}
      )

      text_trace = go.Scatter(
          x=[20], # x-coordinate of the text label
          y=[80], # y-coordinate of the text label
          text=['Gini Coefficient: {:.4f}'.format(df_Gini_Coefficient[df_Gini_Coefficient['Region'] == clicked_category]['Gini Coefficient'].iloc[0])], # Text to display
          mode='text',
          textfont={
              'size': 18,
              'color': 'black'
          }
      )

      # Create a layout for the chart
      layout = go.Layout(
          title=f"Lorenz Curve for {clicked_category}",
          xaxis_title ='Cumulative Population',
          yaxis_title ='Cumulative Income',
          showlegend=False,
          yaxis=dict(range=[0, 120])
      )

      # Create a Figure object and add the traces to it
      fig = go.Figure(data=[bar_trace,text_trace], layout=layout)

      # Show the chart
      return fig

@page.callback(
    Output("viz2a", "figure"),
    [Input("checklist_1", "value")]
)

def update_graph_2(checklist):
  df = df_Gini_Coefficient_2a
  L_Keys = ['REGION I - ILOCOS REGION',
          'REGION II - CAGAYAN VALLEY',
          'REGION III - CENTRAL LUZON',
          'REGION IVA - CALABARZON',
          'REGION V - BICOL REGION',
          'NCR',
          'CAR',
          'MIMAROPA']

  V_Keys = ['REGION VI - WESTERN VISAYAS',
          'REGION VII - CENTRAL VISAYAS',
          'REGION VIII - EASTERN VISAYAS']

  M_Keys = ['REGION IX - ZAMBOANGA PENINSULA',
          'REGION X - NORTHERN MINDANAO',
          'REGION XI - DAVAO REGION',
          'REGION XII - SOCCSKSARGEN',
          'REGION XIII - CARAGA',
          'BARMM']

  LV_Keys = ['REGION I - ILOCOS REGION',
          'REGION II - CAGAYAN VALLEY',
          'REGION III - CENTRAL LUZON',
          'REGION IVA - CALABARZON',
          'REGION V - BICOL REGION',
          'NCR',
          'CAR',
          'MIMAROPA',
          'REGION VI - WESTERN VISAYAS',
          'REGION VII - CENTRAL VISAYAS',
          'REGION VIII - EASTERN VISAYAS']

  LM_Keys = ['REGION I - ILOCOS REGION',
          'REGION II - CAGAYAN VALLEY',
          'REGION III - CENTRAL LUZON',
          'REGION IVA - CALABARZON',
          'REGION V - BICOL REGION',
          'NCR',
          'CAR',
          'MIMAROPA',
          'REGION IX - ZAMBOANGA PENINSULA',
          'REGION X - NORTHERN MINDANAO',
          'REGION XI - DAVAO REGION',
          'REGION XII - SOCCSKSARGEN',
          'REGION XIII - CARAGA',
          'BARMM']

  VM_Keys = ['REGION VI - WESTERN VISAYAS',
          'REGION VII - CENTRAL VISAYAS',
          'REGION VIII - EASTERN VISAYAS',
          'REGION IX - ZAMBOANGA PENINSULA',
          'REGION X - NORTHERN MINDANAO',
          'REGION XI - DAVAO REGION',
          'REGION XII - SOCCSKSARGEN',
          'REGION XIII - CARAGA',
          'BARMM']

  sorted = df.sort_values(by='Gini Coefficient').index
  region_dict = dict(zip(sorted,range(0,len(sorted))))
  Average_Gini = round(df['Gini Coefficient'].mean(),4)

  if 'Luzon' in checklist and len(checklist) == 1:
      L_Index = []
      for key in region_dict:
        if key in L_Keys:
          L_Index.append(region_dict[key])
      L = sorted[L_Index]
      df = df.loc[L]
      colors = ['#fcb97d','#fcb97d','#fcb97d','#fcb97d','#fcb97d','#fcb97d','#fcb97d','#fcb97d']

      trace_1 = go.Bar(x=df['Gini Coefficient'],
                      y=df.index,
                      orientation='h',
                      marker={'color':colors},
                      hovertemplate='Gini Coefficient: %{x:.4f}<extra></extra>',
                      name='Luzon')
      data = [trace_1]
      layout = {
        'title': 'Gini Coefficients for Regions in Luzon',
        'yaxis': {'categoryorder': 'array',
                  'categoryarray': L,
                  'title': 'Regions'},
        'shapes': [{
            'type': 'line',
            'x0': Average_Gini,
            'y0': 0,
            'x1': Average_Gini,
            'y1': len(L)-1,
            'line': {
                'color': 'black',
                'width': 2}}],
        'annotations': [{
            'x': Average_Gini,
            'y': len(L) + 0.1,
            'text': f"Average Gini Coefficient: {Average_Gini:.4f}",
            'showarrow': False,
            'font': {'size': 16, 'color': 'black'}}],
        'xaxis': {'range': [0.53, 0.62],
                  'title': 'Gini Coefficients'}
              }
      fig = go.Figure(data=data, layout=layout)
      pass

  elif 'Visayas' in checklist and len(checklist) == 1:
      V_Index = []
      for key in region_dict:
        if key in V_Keys:
          V_Index.append(region_dict[key])
      V = sorted[V_Index]
      df = df.loc[V]
      colors = ['#a0b9c6','#a0b9c6','#a0b9c6']

      trace_1 = go.Bar(x=df['Gini Coefficient'],
                      y=df.index,
                      orientation='h',
                      marker={'color':colors},
                      hovertemplate='Gini Coefficient: %{x:.4f}<extra></extra>')
      data = [trace_1]
      layout = {
          'title': 'Gini Coefficients for Regions in Visayas',
          'yaxis': {'categoryorder': 'array',
                    'categoryarray': V,
                    'title': 'Regions'},
          'shapes': [{
              'type': 'line',
              'x0': Average_Gini,
              'y0': 0,
              'x1': Average_Gini,
              'y1': len(V)-1,
              'line': {
                  'color': 'black',
                  'width': 2}}],
          'annotations': [{
              'x': Average_Gini,
              'y': len(V) + 0.1,
              'text': f"Average Gini Coefficient: {Average_Gini:.4f}",
              'showarrow': False,
              'font': {'size': 16, 'color': 'black'}}],
          'xaxis': {'range': [0.53, 0.62],
                    'title': 'Gini Coefficients'}
              }
      fig = go.Figure(data=data, layout=layout)

  elif 'Mindanao' in checklist and len(checklist) == 1:
      M_Index = []
      for key in region_dict:
        if key in M_Keys:
          M_Index.append(region_dict[key])
      M = sorted[M_Index]
      df = df.loc[M]
      colors = ['#9a98b5','#9a98b5','#9a98b5','#9a98b5','#9a98b5','#9a98b5']

      trace_1 = go.Bar(x=df['Gini Coefficient'],
                      y=df.index,
                      orientation='h',
                      marker={'color':colors},
                      hovertemplate='Gini Coefficient: %{x:.4f}<extra></extra>')
      data = [trace_1]
      layout = {
          'title': 'Gini Coefficients for Regions in Mindanao',
          'yaxis': {'categoryorder': 'array',
                    'categoryarray': M,
                    'title': 'Regions'},
          'shapes': [{
              'type': 'line',
              'x0': Average_Gini,
              'y0': 0,
              'x1': Average_Gini,
              'y1': len(M)-1,
              'line': {
                  'color': 'black',
                  'width': 2}}],
          'annotations': [{
              'x': Average_Gini,
              'y': len(M) + 0.1,
              'text': f"Average Gini Coefficient: {Average_Gini:.4f}",
              'showarrow': False,
              'font': {'size': 16, 'color': 'black'}}],
          'xaxis': {'range': [0.53, 0.62],
                    'title': 'Gini Coefficients'}
              }
      fig = go.Figure(data=data, layout=layout)

  elif 'Luzon' in checklist and 'Visayas' in checklist and len(checklist) == 2:
      LV_Index = []
      for key in region_dict:
        if key in LV_Keys:
          LV_Index.append(region_dict[key])
      LV = sorted[LV_Index]
      L_Index = []
      for key in region_dict:
        if key in L_Keys:
          L_Index.append(region_dict[key])
      L = sorted[L_Index]
      V_Index = []
      for key in region_dict:
        if key in V_Keys:
          V_Index.append(region_dict[key])
      V = sorted[V_Index]
      df = df.loc[LV]
      df_L = df.loc[L]
      df_V = df.loc[V]
      colors_L = ['#fcb97d','#fcb97d','#fcb97d','#fcb97d','#fcb97d','#fcb97d','#fcb97d','#fcb97d']
      colors_V = ['#a0b9c6','#a0b9c6','#a0b9c6']

      trace_1 = go.Bar(x=df_L['Gini Coefficient'],
                      y=df_L.index,
                      orientation='h',
                      marker={'color':colors_L},
                      hovertemplate='Gini Coefficient: %{x:.4f}<extra></extra>',
                      name='Luzon')
      trace_2 = go.Bar(x=df_V['Gini Coefficient'],
                      y=df_V.index,
                      orientation='h',
                      marker={'color':colors_V},
                      hovertemplate='Gini Coefficient: %{x:.4f}<extra></extra>',
                      name='Visayas')
      data = [trace_1,trace_2]
      layout = {
          'title': 'Gini Coefficients for Regions in Luzon and Visayas',
          'yaxis': {'categoryorder': 'array',
                    'categoryarray': LV,
                    'title': 'Regions'},
          'shapes': [{
              'type': 'line',
              'x0': Average_Gini,
              'y0': 0,
              'x1': Average_Gini,
              'y1': len(LV)-1,
              'line': {
                  'color': 'black',
                  'width': 2}}],
          'annotations': [{
              'x': Average_Gini,
              'y': len(LV) + 0.1,
              'text': f"Average Gini Coefficient: {Average_Gini:.4f}",
              'showarrow': False,
              'font': {'size': 16, 'color': 'black'}}],
          'xaxis': {'range': [0.53, 0.62],
                    'title': 'Gini Coefficients'}
              }
      fig = go.Figure(data=data, layout=layout)

  elif 'Luzon' in checklist and 'Mindanao' in checklist and len(checklist) == 2:
      LM_Index = []
      for key in region_dict:
        if key in LM_Keys:
          LM_Index.append(region_dict[key])
      LM = sorted[LM_Index]
      L_Index = []
      for key in region_dict:
        if key in L_Keys:
          L_Index.append(region_dict[key])
      L = sorted[L_Index]
      M_Index = []
      for key in region_dict:
        if key in M_Keys:
          M_Index.append(region_dict[key])
      M = sorted[M_Index]
      df = df.loc[LM]
      df_L = df.loc[L]
      df_M = df.loc[M]
      colors_L = ['#fcb97d','#fcb97d','#fcb97d','#fcb97d','#fcb97d','#fcb97d','#fcb97d','#fcb97d']
      colors_M = ['#9a98b5','#9a98b5','#9a98b5','#9a98b5','#9a98b5','#9a98b5']

      trace_1 = go.Bar(x=df_L['Gini Coefficient'],
                      y=df_L.index,
                      orientation='h',
                      marker={'color':colors_L},
                      hovertemplate='Gini Coefficient: %{x:.4f}<extra></extra>',
                      name='Luzon')
      trace_2 = go.Bar(x=df_M['Gini Coefficient'],
                      y=df_M.index,
                      orientation='h',
                      marker={'color':colors_M},
                      hovertemplate='Gini Coefficient: %{x:.4f}<extra></extra>',
                      name='Mindanao')
      data = [trace_1,trace_2]
      layout = {
          'title': 'Gini Coefficients for Regions in Luzon and Mindanao',
          'yaxis': {'categoryorder': 'array',
                    'categoryarray': LM,
                    'title': 'Regions'},
          'shapes': [{
              'type': 'line',
              'x0': Average_Gini,
              'y0': 0,
              'x1': Average_Gini,
              'y1': len(LM)-1,
              'line': {
                  'color': 'black',
                  'width': 2}}],
          'annotations': [{
              'x': Average_Gini,
              'y': len(LM) + 0.1,
              'text': f"Average Gini Coefficient: {Average_Gini:.4f}",
              'showarrow': False,
              'font': {'size': 16, 'color': 'black'}}],
          'xaxis': {'range': [0.53, 0.62],
                    'title': 'Gini Coefficients'}
              }
      fig = go.Figure(data=data, layout=layout)

  elif 'Visayas' in checklist and 'Mindanao' in checklist and len(checklist) == 2:
      VM_Index = []
      for key in region_dict:
        if key in VM_Keys:
          VM_Index.append(region_dict[key])
      VM = sorted[VM_Index]
      V_Index = []
      for key in region_dict:
        if key in V_Keys:
          V_Index.append(region_dict[key])
      V = sorted[V_Index]
      M_Index = []
      for key in region_dict:
        if key in M_Keys:
          M_Index.append(region_dict[key])
      M = sorted[M_Index]
      df = df.loc[VM]
      df_V = df.loc[V]
      df_M = df.loc[M]
      colors_V = ['#a0b9c6','#a0b9c6','#a0b9c6']
      colors_M = ['#9a98b5','#9a98b5','#9a98b5','#9a98b5','#9a98b5','#9a98b5']


      trace_1 = go.Bar(x=df_V['Gini Coefficient'],
                      y=df_V.index,
                      orientation='h',
                      marker={'color':colors_V},
                      hovertemplate='Gini Coefficient: %{x:.4f}<extra></extra>',
                      name='Visayas')
      trace_2 = go.Bar(x=df_M['Gini Coefficient'],
                      y=df_M.index,
                      orientation='h',
                      marker={'color':colors_M},
                      hovertemplate='Gini Coefficient: %{x:.4f}<extra></extra>',
                      name='Mindanao')
      data = [trace_1,trace_2]
      layout = {
          'title': 'Gini Coefficients for Regions in Visayas and Mindanao',
          'yaxis': {'categoryorder': 'array',
                    'categoryarray': VM,
                    'title': 'Regions'},
          'shapes': [{
              'type': 'line',
              'x0': Average_Gini,
              'y0': 0,
              'x1': Average_Gini,
              'y1': len(VM)-1,
              'line': {
                  'color': 'black',
                  'width': 2}}],
          'annotations': [{
              'x': Average_Gini,
              'y': len(VM) + 0.1,
              'text': f"Average Gini Coefficient: {Average_Gini:.4f}",
              'showarrow': False,
              'font': {'size': 16, 'color': 'black'}}],
          'xaxis': {'range': [0.53, 0.62],
                    'title': 'Gini Coefficients'}
              }
      fig = go.Figure(data=data, layout=layout)

  elif 'Luzon' in checklist and 'Visayas' in checklist and 'Mindanao' in checklist and len(checklist) == 3:
      L_Index = []
      for key in region_dict:
        if key in L_Keys:
          L_Index.append(region_dict[key])
      L = sorted[L_Index]
      V_Index = []
      for key in region_dict:
        if key in V_Keys:
          V_Index.append(region_dict[key])
      V = sorted[V_Index]
      M_Index = []
      for key in region_dict:
        if key in M_Keys:
          M_Index.append(region_dict[key])
      M = sorted[M_Index]
      df = df.loc[sorted]
      df_L = df.loc[L]
      df_V = df.loc[V]
      df_M = df.loc[M]
      colors_L = ['#fcb97d','#fcb97d','#fcb97d','#fcb97d','#fcb97d','#fcb97d','#fcb97d','#fcb97d']
      colors_V = ['#a0b9c6','#a0b9c6','#a0b9c6']
      colors_M = ['#9a98b5','#9a98b5','#9a98b5','#9a98b5','#9a98b5','#9a98b5']
      trace_1 = go.Bar(x=df_L['Gini Coefficient'],
                      y=df_L.index,
                      orientation='h',
                      marker={'color':colors_L},
                      hovertemplate='Gini Coefficient: %{x:.4f}<extra></extra>',
                      name='Luzon')
      trace_2 = go.Bar(x=df_V['Gini Coefficient'],
                      y=df_V.index,
                      orientation='h',
                      marker={'color':colors_V},
                      hovertemplate='Gini Coefficient: %{x:.4f}<extra></extra>',
                      name='Visayas')
      trace_3 = go.Bar(x=df_M['Gini Coefficient'],
                      y=df_M.index,
                      orientation='h',
                      marker={'color':colors_M},
                      hovertemplate='Gini Coefficient: %{x:.4f}<extra></extra>',
                      name='Mindanao')
      data = [trace_1,trace_2,trace_3]
      layout = {
          'title': 'Gini Coefficients for Regions in Luzon, Visayas, and Mindanao',
          'yaxis': {'categoryorder': 'array',
                    'categoryarray': sorted,
                    'title': 'Regions'},
          'shapes': [{
              'type': 'line',
              'x0': Average_Gini,
              'y0': 0,
              'x1': Average_Gini,
              'y1': len(sorted)-1,
              'line': {
                  'color': 'black',
                  'width': 2}}],
          'annotations': [{
              'x': Average_Gini,
              'y': len(sorted) + 0.1,
              'text':f"Average Gini Coefficient: {Average_Gini:.4f}",
              'showarrow': False,
              'font': {'size': 16, 'color': 'black'}}],
          'xaxis': {'range': [0.53, 0.62],
                    'title': 'Gini Coefficients'}
              }
      fig = go.Figure(data=data, layout=layout)

  return fig



if __name__ == '__main__':
  page.run_server(debug=True, port = 9000)

Dash app running on http://127.0.0.1:9000/
